In [1]:
# 이거 import하는거 아직 못외움
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
categories = ['alt.atheism','talk.religion.misc','comp.graphics','sci.space']
train = fetch_20newsgroups(subset='train',remove = ['headers','footers','quotes'],categories = categories)
test = fetch_20newsgroups(subset='test',remove = ['headers','footers','quotes'],categories = categories)

In [3]:
X_train = train.data
y_train = train.target
X_test = test.data
y_test = test.target
train_target_names = train.target_names
test_target_names = test.target_names

In [4]:
tfidf = TfidfVectorizer(max_features =2000,max_df = 0.5,min_df = 5)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
X_train_tfidf.toarray()

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.2224066, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

# Logistic회귀

In [5]:
# sklearn이 제공하는 logistic regression을 사용
from sklearn.linear_model import LogisticRegression

# count vector에 대해 regression을 해서 NB 와 비교
LR_clf = LogisticRegression()
LR_clf.fit(X_train_tfidf,y_train)

print("train_score",LR_clf.score(X_train_tfidf,y_train))
print("test_score",LR_clf.score(X_test_tfidf,y_test))


train_score 0.9296951819075713
test_score 0.7339246119733924


* 나이브 베이즈보다 성능이 떨어지는 것을 확인할 수 있다. 
    * 첫째 이유는 나이브 베이즈의 가정이 텍스트 분류 환경과 잘맞고
    * 로지스틱 회귀 분석에서 과적합이 일어났기 때문이다.
    
* 과적합을 방지하는 방법으로는 두가지 방법이 있다.
    * 첫째 특성의 수를 줄이는 것
    * 정규화를 이용해 각 특성에 대한 계수가 지나치게 커지는 것을 막는 방법
        * Ridge회귀(L2(제곱)회귀)와 Lasso회귀(L1(절대값)회귀)

# 릿지회귀

In [6]:
# 사이킷런의 RidgeClassifier 이용
from sklearn.linear_model import RidgeClassifier

# 릿지 분류기 선언
ridge_clf = RidgeClassifier() # ... 이 부분을 완성하시오
# 학습
ridge_clf.fit(X_train_tfidf, y_train) # ... 이 부분을 완성하시오

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.960
Test set score: 0.735


In [8]:
def top10_features(classifier, vectorizer, categories):
    # 단어집합을 array로 만들어놓음
    feature_names = np.array(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        # 역순으로 정렬하기 위해 계수에 음수를 취해서 정렬 후 앞에서부터 10개의 값을 반환
        # categories와 naive classifier에서는 각각의 count vector의 feature들의 계수
        top10 = np.argsort(-classifier.coef_[i])[:10] # 내림차순 정렬 해서 상위 10개 뽑음
        # 카테고리에 영향이 큰 특성 10개를 출력
        print("{0}: {1}".format(category, ", ".join(feature_names[top10])))

In [10]:
# def top10_features(classifier, vectorizer, categories): 를 이용하여 top10 feature를 확인하시오
top10_features(ridge_clf, tfidf, train.target_names) # ... 이 부분을 완성하시오

alt.atheism: bobby, religion, atheists, atheism, motto, punishment, satan, deletion, islamic, liar
comp.graphics: graphics, computer, 3d, file, 42, hi, image, using, screen, looking
sci.space: space, orbit, spacecraft, sci, moon, funding, nasa, 23, engineering, nick
talk.religion.misc: christian, blood, christians, fbi, order, objective, hudson, children, abortion, dead


In [11]:
ridge_clf.coef_

array([[-0.13653463,  0.27392561, -0.17542034, ..., -0.5976059 ,
        -0.13816128, -0.08897165],
       [ 0.18961568, -0.17999103,  0.71643096, ..., -0.02634002,
        -0.07464021,  0.21999342],
       [ 0.16238766,  0.01501701, -0.44297895, ..., -0.29413289,
         0.08429874, -0.14569065],
       [-0.21589384, -0.10933962, -0.09857503, ...,  0.9168149 ,
         0.12875138,  0.01548144]])

* 과적합이 더 심해졌음을 확인할 수 있다.
    * 이를 해결하기 위해선 최적의 alpha값을 선택하여야 하는데, 이를 위해서 검증 데이터셋을 도입할 필요가 있다.

In [12]:
from sklearn.model_selection import train_test_split

# 8:2의 비율로 나누기
X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42) # ... 이 부분을 완성하시오

In [13]:
# ... 이 부분을 완성하시오 (전체 코드 작성)
# 설명: train에 대해 학습하고, validation에 대한 score를 재는 코드를 작성하시오.

# 1. alpha=0.1인 분류기 선언
ridge_clf = RidgeClassifier(alpha = 0.1)
# 2. X_train_ridge에 대해 학습
ridge_clf.fit(X_train_ridge,y_train_ridge)
# 3. X_val_ridge에 대해 score 확인
score = ridge_clf.score(X_val_ridge,y_val_ridge)
score

0.7248157248157249

In [15]:
# ... 이 부분을 완성하시오 (전체 코드 작성)
# 설명: alpha를 0.1부터 10까지 0.1씩 증가시켜 학습시키고, score를 확인하는 코드를 작성하시오.
 # (hint: np.arange(start, stop, step))
# 위의 코드에서 alpha=0.1을 변경시켜가면서 score 값을 확인
import numpy as np
# 1. for 문 작성
for alpha in np.arange(0.1, 10, 0.1):
    # 2. 위의 코드 3줄 복사
    ridge_clf = RidgeClassifier(alpha = alpha)
    # 2. X_train_ridge에 대해 학습
    ridge_clf.fit(X_train_ridge,y_train_ridge)
    # 3. X_val_ridge에 대해 score 확인
    score = ridge_clf.score(X_val_ridge,y_val_ridge)
    print(score)


0.7248157248157249
0.7714987714987716
0.7862407862407862
0.800982800982801
0.8058968058968059
0.800982800982801
0.8034398034398035
0.8083538083538083
0.8083538083538083
0.8132678132678133
0.8157248157248157
0.8206388206388207
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8255528255528255
0.8255528255528255
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8206388206388207
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8157248157248157
0.8157248157248157
0.8157248157248157
0.8157248157248157
0.8108108108108109
0.8108108108108109
0.8108108108108109
0.8108108108108109
0.8108108108108109
0.8083538083538083
0.8083538083538083
0.8034398034398035
0.800982800982801
0.800982800982801
0.7985257985257985
0.7985257985257985
0.7985257985257985
0.7985257985257985
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.8034398034398035
0.8034398034398035
0.800

In [16]:
# ... 이 부분을 완성하시오 (전체 코드 작성)
# alpha를 0.1부터 10까지 0.1씩 증가시키면서, score 값을 최대로 만드는 alpha 값을 max_alpha에 저장하시오
# 위의 코드에서 score 값을 비교하여 저장하는 코드 추가

# 1. max_alpha 초기값 선언
max_alpha = 0
max_score = 0
# 2. for 문 작성
    # 3. 위의 코드 3줄 복사
    # 4. if 문을 통한 비교
        # 5. max_alpha 저장

for alpha in np.arange(0.1, 10, 0.1):
    # 2. 위의 코드 3줄 복사
    ridge_clf = RidgeClassifier(alpha = alpha)
    # 2. X_train_ridge에 대해 학습
    ridge_clf.fit(X_train_ridge,y_train_ridge)
    # 3. X_val_ridge에 대해 score 확인
    score = ridge_clf.score(X_val_ridge,y_val_ridge)
    if score>max_score:
        max_alpha = alpha
        max_score = score

print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

Max alpha 1.600 at max validation score 0.826


* 위의 그리드 서치 방식으로 최적의 알파값을 구했으므로 이를 이용해서 test데이터 셋에 대해서 최종 정확도를 계산한다.

In [17]:
# 릿지 분류기 선언
# 위의 셀에서 얻은 최대 alpha 값을 이용하시오
ridge_clf = RidgeClassifier(alpha=1.6) # ... 이 부분을 완성하시오
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.948
Test set score: 0.739


In [19]:
top10_features(ridge_clf, tfidf, train.target_names)

alt.atheism: bobby, religion, atheism, atheists, motto, punishment, islam, deletion, islamic, satan
comp.graphics: graphics, computer, 3d, file, image, hi, 42, using, screen, looking
sci.space: space, orbit, nasa, spacecraft, moon, sci, launch, flight, funding, idea
talk.religion.misc: christian, christians, fbi, blood, order, jesus, objective, children, christ, hudson


## 라쏘회귀
-> 라쏘회귀는 사이킷런의 LogisticRegression에서 `penalty`파라미터를 `l1`으로 지정함으로써 구현할 수 있다.<br>
-> 동시에 정규화정도인 C를 선택해야하고<br>
-> solver = 'liblinear'를 선택해줘야 한다.<br>
* 라쏘회귀는 어떤 특성의 계수가 0에 가까워지면 이를 완전히 0으로 바꾼다는 점에서 차이가 있다
* 하지만 라쏘는 정규화를 통해 과적합을 줄이지만 동시에 특성의 수도 줄어들기 때문에 성능이 향상된다고 보기는 어렵다.

**Lasso와 같이 특성의 수를 줄이는 것을 특성 선택이라고 한다. 특성을 줄임으로써 얻는 장점은 연관성이 떨어지는 특성을 배제하고 중요한 특성을 집중할 수 있다는 것이다.**

In [20]:
lasso_clf= LogisticRegression(penalty='l1',solver = 'liblinear',C=1)
lasso_clf.fit(X_train_tfidf,y_train)
print("train: ",lasso_clf.score(X_train_tfidf,y_train))
print("test: ",lasso_clf.score(X_test_tfidf,y_test))

train:  0.8185840707964602
test:  0.7235772357723578


## 결정트리를 이용한 기타 문서 분류방법
* 사이킷런은 결정트리를 위한 DecisionClassifier클래스와, RandomForestClassifier클래스, 그래디언트 부스팅을 지원하는 GradientBoostingClassifier를 지원한다.
* 결정트리는 학습데이터에 과적합이 되는 성향이 매우 강한데, 문서분류의 겨우 전반적으로 결정트리 기반의 알고리즘이 좋은 성능을 보여주지 못한다.
    * 결정트리의 feature_importances_는 Training set의 column의 순서에 따라서 부여되게 된다.

In [21]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=7)
tree.fit(X_train_tfidf, y_train)

print('#Decision Tree train set score: {:.3f}'.format(tree.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree.score(X_test_tfidf, y_test)))

#Decision Tree train set score: 0.977
#Decision Tree test set score: 0.536


In [22]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=7)
forest.fit(X_train_tfidf, y_train)

print('#Random Forest train set score: {:.3f}'.format(forest.score(X_train_tfidf, y_train)))
print('#Random Forest test set score: {:.3f}'.format(forest.score(X_test_tfidf, y_test)))

#Random Forest train set score: 0.977
#Random Forest test set score: 0.685


In [23]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=7)
gb.fit(X_train_tfidf, y_train)

print('#Gradient Boosting train set score: {:.3f}'.format(gb.score(X_train_tfidf, y_train)))
print('#Gradient Boosting test set score: {:.3f}'.format(gb.score(X_test_tfidf, y_test)))

#Gradient Boosting train set score: 0.933
#Gradient Boosting test set score: 0.696


In [24]:
gb.feature_importances_

array([0.        , 0.        , 0.00018793, ..., 0.00321606, 0.        ,
       0.        ])

In [25]:
sorted_feature_importances = sorted(zip(tfidf.get_feature_names_out(), gb.feature_importances_), key=lambda x: x[1], reverse=True)
for feature, value in sorted_feature_importances[:40]:
    print('%s: %.3f' % (feature, value), end=', ')

space: 0.126, graphics: 0.080, atheism: 0.024, thanks: 0.023, file: 0.021, orbit: 0.020, jesus: 0.018, god: 0.018, hi: 0.017, nasa: 0.015, image: 0.015, files: 0.014, christ: 0.010, moon: 0.010, bobby: 0.010, launch: 0.010, christian: 0.010, looking: 0.010, atheists: 0.009, christians: 0.009, fbi: 0.009, 3d: 0.008, you: 0.008, not: 0.008, islamic: 0.007, religion: 0.007, spacecraft: 0.007, flight: 0.007, computer: 0.007, islam: 0.007, ftp: 0.006, color: 0.006, software: 0.005, atheist: 0.005, card: 0.005, people: 0.005, koresh: 0.005, his: 0.005, kent: 0.004, sphere: 0.004, 

# [[실습 4-2-1]]
# coef_의 결과와 feature_importances_ 결과 비교를 통해, 회귀분석 방법과 트리 기반의 분류 방식의 차이점 알아보기
* coef_의 결과(여러 분류기 중 1개 이상)와 feature_importances_ 결과(여러 분류기 중 1개 이상) 복사/붙여넣기
* 비교를 통해 어떤 차이점이 있는 것 같은지 분석해보시오.
> 결정트리에서는 성격이 비슷한 두 단어 중 하나가 분류에 먼저 사용되면 다른 단어는 상대적으로 중요도가 떨어지게 된다.

## 한국어 문서의 분류
* 영어 문서는 사이킷런에서 제공하는 기본 토크나이저(CountVectorizer,TfidfVectorizer)를 사용할 수 있었으나, 한글 문서는 처음부터 별도의 토크나이저를 쓸 수 밖에 없다.

### 입력 값으로 리뷰가 들어오면, 어떤 영화에 대한 리뷰인지 예측하는 알고리즘을 만들어 보자!
**&rightarrow; 앞에서는 영화리뷰를 통해서 영화간의 유사도를 비교하였지만, 여기서는 이제 영화리뷰를 이용해서 영화의 제목을 예측하는 분류기를 만들어보자**
* '신과함께, 택시운전사, 인피니티 워, 범죄도시, 곤지암, 라라랜드, 코코' 중 하나로 분류
* **입력**: 몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.
* **출력**: 인피니티 워

In [26]:
import pandas as pd
df = pd.read_csv('./daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [27]:
df.title.value_counts()

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

In [28]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
# 비율을 지정하지 않으면 75:25로 분할됨
# train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)를 참고
X_train, X_test, y_train, y_test = train_test_split(df.review,df.title, random_state=0) # ... 이 부분을 완성하시오

print('#Train set size:', len(X_train)) # 실제로 몇개의 특성이 사용되었는지 확인
print('#Test set size:', len(X_test))

#Train set size: 11043
#Test set size: 3682


In [29]:
from konlpy.tag import Okt # konlpy에서 Twitter 형태소 분석기를 import

okt = Okt()

print(okt.morphs(X_train[1])) # 둘째 리뷰에 대해 형태소 단위로 tokenize
print(okt.nouns(X_train[1])) # 둘째 리뷰에서 명사만 추출

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


* 일반적으로 문서를 대상으로 분류를 하는 경우에는 명사만으로도 좋은 결과를 보이는 경우가 많으므로 우선 Okt.nouns를 tokenizer로 사용해서 카운트벡터를 만들어보자.

In [30]:
# 약간의 시간이 소요됩니다! (약 1분 30초)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Okt 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
# mytokenizer를 okt.nouns로 여기선 사용함
# mytokenizer를 사용하면 그때 그때 호출해야해서오래걸림
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

In [31]:
X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

In [32]:
# logistic regression 분류기 선언
# 충분한 학습을 위해서 max_iter=1000으로 설정
# 기본값은 100인데 100으로 돌리면 충분히 수렴되지 않았다는 warning이 발생하게 된다.
LR_clf = LogisticRegression(max_iter = 1000)
# 분류기 학습
LR_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data 예측정확도
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data 예측정확도

#Train set score: 0.756
#Test set score: 0.694


In [35]:
top10_features(LR_clf, tfidf, sorted(df.title.unique()))

곤지암: 공포영화, 공포, 귀신, 곤지암, 무서움, 소리, 기담, 개, 공포물, 장난
라라랜드: 음악, 뮤지컬, 꿈, 사랑, 노래, 엔딩, 여운, 인생, 재즈, 환상
범죄도시: 마동석, 윤계상, 조선족, 액션, 조연, 조폭, 범죄, 마블리, 형사, 폭력
신과함께: 원작, 차태현, 웹툰, 신파, 김동욱, 판타지, 신, 천만, 지옥, 하정우
인피니티 워: 마블, 노스, 어벤져스, 히어로, 번역, 토르, 편, 결말, 다음, 영웅
코코: 디즈니, 애니, 겨울왕국, 감동, 코코, 픽사, 애니메이션, 가족, 어른, 노래
택시운전사: 광주, 송강호, 역사, 전두환, 택시, 가슴, 진실, 민주주의, 감동, 추격


In [36]:
# 첫 번째 리뷰에 대한 예측 값 살펴보기
print('첫 번째 리뷰 텍스트: ', X_test.iloc[0])
print('특성으로 추출된 토큰: ', okt.nouns(X_test.iloc[0]))
predict = LR_clf.predict(X_test_tfidf[0])
print('예측된 영화 제목: ', predict[0])
print('실제 영화 제목: ', y_test.iloc[0])

첫 번째 리뷰 텍스트:  오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.
특성으로 추출된 토큰:  ['만', '영화', '다음', '더', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  범죄도시


In [33]:
LR_clf.predict(X_test_tfidf)

array(['신과함께', '범죄도시', '코코', ..., '인피니티 워', '인피니티 워', '신과함께'],
      dtype=object)

In [34]:
print('실제 영화 제목, 예측한 제목, 리뷰')
for content in zip(y_test[:10],LR_clf.predict(X_test_tfidf)[:10],X_test[:10]):
    print(content)

실제 영화 제목, 예측한 제목, 리뷰
('범죄도시', '신과함께', '오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.')
('범죄도시', '범죄도시', '조연들이 눈에 박힌다. 간만에 집중 ㅎ')
('코코', '코코', '대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??')
('신과함께', '신과함께', '돈이 안아까웠던 영화ᆞᆞ  정말 좋았다')
('신과함께', '신과함께', '역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.')
('택시운전사', '택시운전사', '민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.')
('신과함께', '신과함께', '잠만 자다 왔음')
('신과함께', '신과함께', '오랜만에 잼있고 좋은 영화를 봤다')
('범죄도시', '신과함께', '잼남')
('범죄도시', '인피니티 워', '대박~~')


/tmp/ipykernel_839936/568123338.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for content in zip(y_test[:10],LR_clf.predict(X_test_tfidf)[:10],X_test[:10]):


# [[실습 4-2-2]]
* RidgeClassifier (alpha=0.1)를 이용하여 분류할 경우, Train set score와 Test set score는 얼마인가?
* (가능한 경우에만 시도) 이 데이터 셋에서는 valdation에 대한 최적의 alpha는 몇인가? 0.1과 10 사이에서 0.1씩 증가시키며 찾아보시오.


In [38]:
# 코드를 작성하시오
ridge_clf = RidgeClassifier(alpha = 0.1)
ridge_clf.fit(X_train_tfidf,y_train)
print("train score: {:.3f}".format(ridge_clf.score(X_train_tfidf,y_train)))
print("test score:{:.3f}".format(ridge_clf.score(X_test_tfidf,y_test)))

train score: 0.788
test score:0.679


In [39]:
from sklearn.model_selection  import train_test_split
X2_train,X_val,y2_train,y_val = train_test_split(X_train_tfidf,y_train,test_size = 0.2,random_state = 42)
optim_alpha = 0
max_score = 0
for alpha in np.arange(0.1,10,0.1):
    ridge_clf = RidgeClassifier(alpha = alpha)
    ridge_clf.fit(X2_train,y2_train)
    score = ridge_clf.score(X_val,y_val)
    if score>max_score:
        optim_alpha = alpha
        max_score = score

print("최적 alpha",optim_alpha)
print("최적 score",max_score)


최적 alpha 1.1
최적 score 0.6962426437301946
